## 2.1 Processes ##
1. Process: an abstraction of a running program.

2. Pseudoparallelism v.s. multiprocessor 

3. In each CPU, there is always one process running.

### 2.1.1 The Process Model ###
1. In this model, all the runnable software on the computer, sometimes including the operating system, is organized into a number of ***sequential processes(processes)***.

2. A process is an instance of an executing program, including the current values of the program counter, registers, and variables. Conceptually, each process has its own virtual CPU. 

3. Processes must not be programmed with built-in assumptions about timing.

4. Process v.s. program
        - a ***process*** is an activity of some kind. It has a program, input, output, and a state.
        - a single processor may be shared among several processes, with some scheduling algorithm being accustomed to determine when to stop work on one proces and service a different one.
        - in contrast, a ***program*** is something that may be stored on disk, not doing anything.

### 2.1.2 Process Creation ###
1. Four principle events cause processes to e created:
        - System initialization.
        - Execution of a process-creation system call by a running process.
        - A user request to create a new process.
        - Initiaiton of a batch job.
        
2. Deamons: processes that stey in the background to handle some activity.

3. Technically, in all cases, a new process is created by having an existing process execute a process creation system call.

4. `fork` in the only one system call to create a new process in UNIX. This call creates an exact clone fo the calling process. After that, the child process executes `execve` or a similar system call to change its memory image and run a new program.

5. In Windows, in contrast, s single Win32 function call, `CreateProcess`, handles both process creation and loading the correct program into the new process.

6. In both UNIX and Windows systems, after a process is created, the parent and child have their own distinct address spaces. NO WRITABLE MEMORY IS SHARED.


### 2.1.3 Process Termination ###
1. Conditions where new process will terminate:
        - Normal exit (voluntary). E.g. the process has done its work
        - Error exit (voluntary). E.g. the process discovers a fatal error
        - Fatal error (involuntary). E.g. caused by the process, bugs
        - Killed by another process (involuntary). E.g. (by system call `kill`)
        
### 2.1.4 Process Hierarchies ###
1. In some systems, when a process creates another process, the parent process and child process continue to be associated in certain ways. The child process can itself create more processes, forming a process hierarchy.

2. A process has only one parent but can have multiple chilren.

3. In UNIX, a process and all of its children and further descendants together form a process group.

4. Unlike in Windows, process in UNIX cannot disinherit their children.

### 2.1.5 Process States ###
1. Three states a process may be in:
        - Running (actually using the CPU at that instant).
        - Ready (runnable; temporarily stopped to let another process run).
        - Blocked (unable to run until some external event happends).

2. Four possible transitions among these three states:
        - Process blocks for input.
        - Scheduler picks another paroacess.
        - Scheduler picks this process.
        - Input becomes available.
    
3. The lowest layer of a process-structured operating system handles interrupts and scheuling. Above taht layer are sequential processes.

### 2.1.6 Implementation of Processes ###
1. The operating system maintains a table (an array of structures), called the **process table/process control blocks**. This entry contains import information abotu the process' state, i.e., its program counter, stack pointer, memory allocation, the status of its open files, its accouting and scheduling information.

2. Associated with each I/O class is a location (typically at a fixed location near the bottom of memory) called the **interrupt vector**. It contains the address of the interrupt service procedure. 

### 2.1.7 Modeling Multiprogramming ###
1. Skeleton of what the lowest level of the operating system does when an interrupt occurs:
        a. Hadware stacks program counter, etc.
        b. Hardware loads new program counter from interrupt vector.
        c. Assembly-language procedure saves registers.
        d. Assembly-language procedure sets up new stack.
        e. C interrupt service runs (typically reads and buffers input)
        f. Scheduler decides which process is to run next.
        g. C procedure returns to the assembly code.
        h. Assembly-language procedure starts up new current process.
        
2. Degree of multiprogramming : don't quite get this 

The result of using pop-up threads is that the latency between message arrival and the start of processing can be made very short.

## 2.2 Threads ##
In traditional operating systems, each process has an address space and a single ghread of control. In fact, thatis almost the definition of a process. Nevertheless, in many situations, it is desirable to have multiple threads of control in the same address space running in quasi-parallel, as though they were (almost) separate processes (except for the shared address space).

### 2.2.1 Thread Usage ###
1. Threads: miniprocesses running in a process

2. Reasons to have threads:
        - In many aplications, multiple activities are going on at once. Some of these may block from time to time.
        - Threads have the ability for the parallel entities to share an address space and all of its data among themselves.
        - Threads are lighter weight than processes, thus they are easier (i.e., faster) to create and destroy.
        - Threads yield no performance gain when all of them are CPU bound, but when there is substantial computing and also substantial I/O, having threads allows these activities to overlap, thus speeding up the application.
        - Threads are useful on systems with multiple CPUs, where real parallelism is possible.
        
3. Example: Word processors: one thread listening to user input, one thread reformatting files, and one backup the whole file periodically.

4. Finite-state machine: each computation has a saved state, and there exists some set or events that can occur to change the state.

5. Three ways to construct a server:
        **Model**                  **Characteristics**
        Threads                     Parallelism, blocking system calls
        Single-threaded process     No parallelism, blocking system calls
        Finite-state machine        Parallelism, nonblocking system calls, interrupts

6. Example: an application that processes very large amounts of data. One input thread, one processing thread, one output thread.

### 2.2.2 The Classical Tread Model ###
1. The process model is based on two independent concepts: resource grouping and execution.

2. One way of looking at a process is that it is a way to group related resources together. A process has an address space containing program text and data, as well as other resources. These resources may include open files, child processes, pending alarms, signal handlers, accouting information, and more. By putting them together in the form of a process, they can be managed more easily.

3. The other concept a process has is a thread of execution. Process are used to group resources together, threads are the entities scheduled for execution on the CPU.

4. What treads add to the process model is to allow multiple executions to take place in the same process enviroment, to a large degree independent of one another.

5. When a multithreaded process is run on a single-CPU system, the threads take turns running.

6. Items shared by all threads in a process and items private to each thread:
        **Per-process items**            **Per-thread items**
        Address space                    Program counter
        Global variables                 Registers
        Open files                       Stack
        Child processes                  State
        Pending alarms
        Signals and signal handlers
        Accouting information
        
7. States of a thread: running; blocked; ready; terminated

8. `thread_create`. When multithreading is present, processes usually start with a single thread present. This ghread ha sthe ability to create new threads by calling this library procedure.

9. `thread_exit`. When a tread finishes its work.

10. `thread_join`. One thread can wait for a (specific) thread to exit by calling this procedure.

11. `thread_yield`. Allows a thread to voluntarily give up the CPU to let another thread run.

### 2.2.3 POSIX Threads ###
1. Pthreads: To make it possible to write portable threaded programs, IEEE has defined a standard to threads in IEEE standard 1003.1c. The threads package it defines is called **Pthreads**.

### 2.2.4 Implementing Threads in User Space ###
1. Advantages: 
        - A user-level threads package can be implemented on an operating system that does not support threads.
        - They allow each process to have its own customized scheduling algorithm.

2. When threads are managed in user space, each process needs it own private **thread table** to keep track of the threads in that process.

3. If the program calls or jumps to an instruction that is not in memory, a **page fault** occurs and the operating system will go and get the missing instruction (and its neighbors) from disk.

4. Disavantages:
        - The problem of hwo blocking system calls are implemented.
        - If a thread starts runing, no other thread in that process will ever run unless the first thread voluntarily gives up the CPU.
        - Programmers generally want threads precisely in application where the threads block often, as, for example, in a multithreaded Web server.


### 2.2.5 Implementing Threads in the Kernel ###
1. The kernel's thread table holds each thread's registers, state, and other information.

2. All calls that might block a thead are implemented as system calls, at consideraly greater cost than a call to a run-time system procedure.

3. When a thread blocks, the kernel, can run either another thread from the same process (if one is ready) or a thread from a different process.

4. Problems kernel threads can't solve perfectly:
        - What happens when a multithreaded process forks?
        - When a signal comes in (signals are sent to processes, not to threads), which thread should handle it?
        
### 2.2.6 Hybrid Implementation ###
Use kernel-level threads and then multiplex user-level threads onto some or all of them. 

### 2.2.7 Scheduler Activations ###
1. Approach divised by Anderson et al. (1988). The goals of the scheduler activation work are to mimic the functionality of kernal threads, but with the better performance and greater flexibility usually associated with threads packages implemented in user space.

2. User threads should not have to make special nonblocking system calls or check in advance if it is safe to make certain system calls. Nevertheless, when a thread blocks on a system call or on a page fault, it should be possible to run other threads within the same process.

3. When scheduler activations are used, the kernel assigns a certain number of virtual processors to each process and lets the (user-space) run-time system allocate threads to processors. 

4. upcall

### 2.2.8 Pop-Up Threads ###
1. In the example of requests for service. The arrival of a message causes the system to create a new thread to handle the message. Such a thread is called **pop-up thread**.

2. The result of using pop-up threads is that the latency between message arrival and the start of processing can be made very short.

### 2.2.9 Making Single-Threaded Code Multithreaded ###
1. Problems need consideration:
        - private global variables
        - many library procedures are not reentrant (meaning they are not designed to have a second call made to any given procedure while a previous call has not yet finished
        - some signals are logically thread specific, whereas others are not
        - stack management (when a process' stack overflows, the kernel just provides that process with more stack automatically. When a process has multiple threads, it must also have multiple stacks. If the kernel is not aware of all these stacks, it cannot grow automatically upon stack falt.)

## 2.3 Interprocess Communication ##
1. InterProcess Communication, also called IPC.

2. Three main issues:
        - how one process can pass information to another
        - how to make sure two or more processes do not get in each other's way
        - how to deal with proper sequencing when dependencies are present

3. The above issues and solutions also apply to threads.

### 2.3.1 Race Conditions ###
1. ***Race conditions***: situations where two or more processes are reading or writing some shared data and the final result depends on who runs precisely when.

2. Example: print spooler, where process A and B wants to write in the next free slot. Just the minute process A reads and stores the value, a clock interrupt occurs and the CPU decides A has run long enough and witches to process B. Process B read and write into the slot and erase what process A has written.

### 2.3.2 Critical Regions ###
1. ***Mutual exclusion***: some way of making sure that if one process is using a shared variable or file, the other process will be excluded from doing the same thing.

2. ***Critical region/critical section***: the part of the program where the shared memory is accessed.

3. To aviod condition races, we have to make sure no two processes are in their critical regions at the same time.

4. A good solution for race conditions meets the following 4 requirements:
        - No two processes may be simultaneously inside their critical regions.
        - No assumptions may be made about speeds or the number of CPUs.
        - No process running outside its critical region may block any process.
        - No process should have to wait forever to enter its critical region.
        
### 2.3.3 Mutual Exlusion with Busy Waiting ###